In [ ]:
from google.colab import drive
from google.colab import files
import os
drive.mount('/content/drive')
os.chdir('/content')
!nvidia-smi

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2

In [ ]:
!mkdir /content/data
%cd data

from google_drive_downloader import GoogleDriveDownloader as gdd
import os
import re
import shutil
#load dataset
gdd.download_file_from_google_drive(file_id='1nEJ7NTtHcCHNQqUXaoPk55VH3Uwh4QGG',
                                    dest_path='./dataset.zip',
                                    unzip=True)

#this is for annotaion.json from prepare_anno
gdd.download_file_from_google_drive(file_id='1WTxF13cjdoBCgt2hKBpnJ7vbbjtGuQOR',
                                    dest_path='./dataset/annotations.zip',
                                    unzip=True)
def modify_dataset_to_coco():
    
    
    train_names = os.listdir('./dataset/train')
    train_names = list(filter(lambda x: re.match('TCGA', x) != None, train_names))
    
    for tr_name in train_names:
        tr_img_path = os.path.join('dataset', 'train', tr_name, 'images', tr_name+'.png')
        new_tr_img_path = os.path.join('dataset', 'train', tr_name+'.png')
        os.rename(tr_img_path, new_tr_img_path)
    
    for folder_name in train_names:
        shutil.rmtree(f'./dataset/train/{folder_name}', ignore_errors=True)

modify_dataset_to_coco()

%cd /content

/content/data
Unzipping...Done.
Unzipping...Done.
/content


In [ ]:
%cd /content
from detectron2.data.datasets import register_coco_instances

#train:
register_coco_instances('train_custom', {}, '/content/data/dataset/annotations.json', '/content/data/dataset/train')
#test:
register_coco_instances('test_custom', {}, '/content/data/dataset/test_img_ids.json', '/content/data/dataset/test')


In [ ]:
import argparse
import sys
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, MetadataCatalog, build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import CityscapesSemSegEvaluator, DatasetEvaluators, SemSegEvaluator
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import detection_utils as utils
from detectron2 import model_zoo
from detectron2.projects import point_rend
import os

cfg = get_cfg()
# point_rend.add_pointrend_config(cfg) # it you want to train pointrend please unmark this line

# then you have some yaml to choose
# pointrend or resnet...
# cfg.merge_from_file("detectron2_repo/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml")
# cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
# cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
# cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.INPUT.FLIP = 'horizontal'
cfg.DATASETS.TRAIN = ("train_custom")
cfg.DATALOADER.NUM_WORKERS = 4

# choose the corresponding weight
# cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/FAIR/X-101-32x8d.pkl"
# cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
# cfg.MODEL.WEIGHT = "detectron2://ImageNetPretrained/MSRA/R-101.pkl"

# if you use pointrend this line should be marked
# cfg.MODEL.POINT_HEAD.NUM_CLASSES = 1

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ANCHOR_GENERATOR.SIZES =[[8], [16], [32], [64], [128]]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 1000

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025         # Base learning rate
# cfg.SOLVER.GAMMA = 0.5          # Learning rate decay
cfg.SOLVER.STEPS = (500, 1000, 1500)    # Iterations at which to decay learning rate
cfg.SOLVER.WARMUP_ITERS = 100        # Warmup iterations to linearly ramp learning rate from zero
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.DATASETS.TEST = ()

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(False)
trainer.train()


[12/13 04:21:12 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[12/13 04:21:14 d2.engine.train_loop]: Starting training from iteration 0
[12/13 04:21:34 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f87ccadd3b0> to CPU due to CUDA OOM
[12/13 04:21:57 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f87ccadd3b0> to CPU due to CUDA OOM
[12/13 04:22:25 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f87ccadd3b0> to CPU due to CUDA OOM
[12/13 04:22:49 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f87ccadd3b0> to CPU due to CUDA OOM
[12/13 04:23:18 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f87ccadd3b0> to CPU due to CUDA OOM
[12/13 04:23:35 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f87ccadd3b0> to CPU due to CUDA OOM
[12/13 04:24:16 d2.utils.events]:  eta: 2:17:24  iter: 19  total_loss: 1.548  loss_cls: 0.2537  loss_box_reg: 0.3051  loss_mask: 0.4406  loss_rpn_cls: 0.3246 

this part try to use different data augmentation skills

In [ ]:
import argparse
import sys
from detectron2.data import transforms as T
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, MetadataCatalog, build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import CityscapesSemSegEvaluator, DatasetEvaluators, SemSegEvaluator
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import detection_utils as utils
from detectron2 import model_zoo
from detectron2.projects import point_rend
import os

def build_sem_seg_train_aug(cfg):
    augs = [
        T.ResizeShortestEdge(
            cfg.INPUT.MIN_SIZE_TRAIN, cfg.INPUT.MAX_SIZE_TRAIN, cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING
        ),
        T.RandomBrightness(0.8, 1.5),
        T.RandomContrast(0.8, 1.5),
        T.RandomFlip(0.5, horizontal=True),
        T.RandomFlip(0.5, horizontal=False, vertical=True),
        T.RandomSaturation(0.8, 1.2),
        # T.RandomRotation(-30, 30),
    ]
    return augs

class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains a number pre-defined logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can use the cleaner
    "SimpleTrainer", or write your own training loop.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
        if evaluator_type == "sem_seg":
            return SemSegEvaluator(
                dataset_name,
                distributed=True,
                output_dir=output_folder,
            )
        if evaluator_type == "cityscapes_sem_seg":
            assert (
                torch.cuda.device_count() > comm.get_rank()
            ), "CityscapesEvaluator currently do not work with multiple machines."
            return CityscapesSemSegEvaluator(dataset_name)
        if len(evaluator_list) == 0:
            raise NotImplementedError(
                "no Evaluator for the dataset {} with the type {}".format(
                    dataset_name, evaluator_type
                )
            )
        if len(evaluator_list) == 1:
            return evaluator_list[0]
        return DatasetEvaluators(evaluator_list)

    @classmethod
    def build_train_loader(cls, cfg):
        if "GeneralizedRCNN" in cfg.MODEL.META_ARCHITECTURE:
            mapper = DatasetMapper(cfg, is_train=True, augmentations=build_sem_seg_train_aug(cfg))
        else:
            mapper = None
        return build_detection_train_loader(cfg, mapper=mapper)


def setup(args, is_train=True):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    
    cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

    cfg.INPUT.MIN_SIZE_TRAIN = 900
    cfg.INPUT.MAX_SIZE_TRAIN = 1300
    cfg.DATASETS.TRAIN = ("train_custom",)
    cfg.DATASETS.TEST = ("test_custom",)
    cfg.DATALOADER.NUM_WORKERS = 4

    cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/VRDL/hw3/json_file/(10)r50_fpn_0.00025_1e3_anchor8_na.pth"
    
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.ANCHOR_GENERATOR.SIZES =[[8], [16], [32], [64], [128]]
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256

    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.BASE_LR = 0.00025         # Base learning rate
    # cfg.SOLVER.GAMMA = 0.5              # Learning rate decay
    cfg.SOLVER.STEPS = (300, 500, 700)  # Iterations at which to decay learning rate
    cfg.SOLVER.WARMUP_ITERS = 100       # Warmup iterations to linearly ramp learning rate from zero
    cfg.SOLVER.MAX_ITER = 800
    
    trainer = Trainer(cfg)
    trainer.resume_or_load(False)
    trainer.train()

    cfg.freeze()
    default_setup(cfg, args)
    return cfg

def default_argument_parser(epilog=None):
    """
    Create a parser with some common arguments used by detectron2 users.

    Args:
        epilog (str): epilog passed to ArgumentParser describing the usage.

    Returns:
        argparse.ArgumentParser:
    """
    parser = argparse.ArgumentParser(
        epilog=epilog
        or f"""
Examples:

Run on single machine:
    $ {sys.argv[0]} --num-gpus 8 --config-file cfg.yaml

Change some config options:
    $ {sys.argv[0]} --config-file cfg.yaml MODEL.WEIGHTS /path/to/weight.pth SOLVER.BASE_LR 0.001

Run on multiple machines:
    (machine0)$ {sys.argv[0]} --machine-rank 0 --num-machines 2 --dist-url <URL> [--other-flags]
    (machine1)$ {sys.argv[0]} --machine-rank 1 --num-machines 2 --dist-url <URL> [--other-flags]
""",
        formatter_class=argparse.RawDescriptionHelpFormatter,
    )
    parser.add_argument("--config-file", default="", metavar="FILE", help="path to config file")
    parser.add_argument(
        "--resume",
        action="store_true",
        help="Whether to attempt to resume from the checkpoint directory. "
        "See documentation of `DefaultTrainer.resume_or_load()` for what it means.",
    )
    parser.add_argument("--eval-only", action="store_true", default=False, help="perform evaluation only")
    parser.add_argument("--num-gpus", type=int, default=1, help="number of gpus *per machine*")
    parser.add_argument("--num-machines", type=int, default=1, help="total number of machines")
    parser.add_argument(
        "--machine-rank", type=int, default=0, help="the rank of this machine (unique per machine)"
    )

    # PyTorch still may leave orphan processes in multi-gpu training.
    # Therefore we use a deterministic way to obtain port,
    # so that users are aware of orphan processes by seeing the port occupied.
    port = 2 ** 15 + 2 ** 14 + hash(os.getuid() if sys.platform != "win32" else 1) % 2 ** 14
    parser.add_argument(
        "--dist-url",
        default="tcp://127.0.0.1:{}".format(port),
        help="initialization URL for pytorch distributed backend. See "
        "https://pytorch.org/docs/stable/distributed.html for details.",
    )
    parser.add_argument(
        "opts",
        help="""
Modify config options at the end of the command. For Yacs configs, use
space-separated "PATH.KEY VALUE" pairs.
For python-based LazyConfig, use "path.key=value".
        """.strip(),
        default=None,
        nargs=argparse.REMAINDER,
    )
    return parser
def main(args):
    cfg = setup(args)

    if args.eval_only:
        model = Trainer.build_model(cfg)
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        res = Trainer.test(cfg, model)
        return res

    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=args.resume)
    return trainer.train()


if __name__ == "__main__":
    args = default_argument_parser().parse_args(args=[])
    print("Command Line Args:", args)
    launch(
        main,
        args.num_gpus,
        num_machines=args.num_machines,
        machine_rank=args.machine_rank,
        dist_url=args.dist_url,
        args=(args,),
    )

this part aims to generate submission and save model weight

In [ ]:
def get_dicts(img_dir):
    TEST_SIZE = 6
    json_file_path = os.path.join(img_dir, "test_img_ids.json")
    with open(json_file_path) as f:
        info = json.load(f)
    filenames = []
    imgids = []
    for i in range(TEST_SIZE):
        filenames.append(info[i]['file_name'])
        imgids.append(info[i]['id'])
    for img, file in zip(imgids, filenames):
        yield img, file


In [ ]:
from detectron2.utils.visualizer import ColorMode
from detectron2.engine import DefaultPredictor
from PIL import Image
import json
import cv2
import numpy as np
import torch
import pycocotools._mask as _mask
# if use custom data augmentation:
# cfg = get_cfg()
# cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05

cfg.DATASETS.TEST = ("test_custom")
cfg.DATALOADER.NUM_WORKERS = 4

cfg.MODEL.WEIGHTS = '/content/output/model_final.pth'

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ANCHOR_GENERATOR.SIZES =[[8], [16], [32], [64], [128]]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128

predictor = DefaultPredictor(cfg)

final_list = []
for id, file in get_dicts('/content/data/dataset/'):
    im = cv2.imread(f'/content/data/dataset/test/{file}') 
    outputs = predictor(im)

    field_list = outputs['instances'].get_fields()
    bbox_all = field_list['pred_boxes'].tensor
    score_all = field_list['scores']
    seg_all = field_list['pred_masks']

    for ins in range(score_all.shape[0]):
        bbox_ins = bbox_all[ins].tolist()
        score_ins = score_all[ins].item()
        seg_ins = seg_all[ins].cpu()
        seg_ins = np.asfortranarray(seg_ins) 
        h, w = seg_ins.shape
        rle_ins = _mask.encode(seg_ins.reshape((h, w, 1), order='F'))[0] 
        rle_ins['counts'] = rle_ins['counts'].decode('ascii')
        final_list.append(dict(
            image_id = id, 
            bbox = bbox_ins, 
            score =  score_ins, 
            category_id = 1, 
            segmentation = rle_ins
        ))


In [ ]:
from shutil import copyfile
import json
# save submission
with open('/content/drive/MyDrive/VRDL/hw3/json_file/answer.json', 'w') as fp:
    json.dump(final_list, fp, indent=4)
#save model weight
copyfile('/content/output/model_final.pth', '/content/drive/MyDrive/VRDL/hw3/json_file/(8)x101_fpn_0.00025_4e3_anchor8_na.pth')